In [ ]:
from facebook_business.api import FacebookAdsApi
from facebook_business.exceptions import FacebookRequestError
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccountuser import AdAccountUser as AdUser
from facebook_business import adobjects
from pandas import DataFrame
import time

In [ ]:
access_token = 'ACCESS_TOKEN'
ad_account_id = "act_AD_ACCOUNT_ID"
app_secret = 'APP_SECRET'
app_id = 'APP_ID'

FacebookAdsApi.init(access_token=access_token)

In [ ]:
me = AdUser(fbid='me')
my_accounts = list(me.get_ad_accounts())
my_accounts

In [ ]:
my_account = my_accounts[0]
campaigns = my_account.get_campaigns()
print(campaigns)

In [ ]:
my_account.api_get(fields=[AdAccount.Field.amount_spent])
print(int(my_account[AdAccount.Field.amount_spent])/100)

In [ ]:
fields = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.clicks,
    AdsInsights.Field.spend,
    AdsInsights.Field.impressions]

count = 0

In [ ]:
def wait_for_async_job(async_job):
    global count
    async_job = async_job.api_get()
    while async_job[AdReportRun.Field.async_status] != 'Job Completed' or async_job[AdReportRun.Field.async_percent_completion] < 100:
        time.sleep(2)
        async_job = async_job.api_get()
    else:
        print(f"Job {count} completed")
        count += 1
    return async_job.get_result(params={"limit": 1000})

In [ ]:
def get_insights(account, date_preset='last_3d'):
    account = AdAccount(account['id'])
    i_async_job = account.get_insights(
        params={
            'level': 'ad',
            'date_preset': date_preset,
            'time_increment': 1
        },
        fields=fields,
        is_async=True
    )
    results = [dict(item) for item in wait_for_async_job(i_async_job)]
    return results

In [ ]:
elem_insights = []
insights_lists = []
date_preset = 'last_7d'

In [ ]:
for account in my_accounts:
    account_insights = get_insights(account, date_preset)
    insights_lists.append(account_insights)

In [ ]:
insights_camplaign_id_list = []
insights_click_list = []
insights_spend_list = []
insights_impressions_list = []
insights_date_start_list = []
insights_date_stop_list = []

In [ ]:
for account in insights_lists:
    for campaign in account:
        insights_camplaign_id_list.append(int(campaign['campaign_id']))
        insights_click_list.append(int(campaign['clicks']))
        insights_spend_list.append(float(campaign['spend']))
        insights_impressions_list.append(int(campaign['impressions']))
        insights_date_start_list.append(campaign['date_start'])
        insights_date_stop_list.append(campaign['date_stop'])

In [ ]:
df = DataFrame()
df['campaign_id'] = insights_camplaign_id_list
df['clicks'] = insights_click_list
df['spend'] = insights_spend_list
df['impressions'] = insights_impressions_list
df['date_start'] = insights_date_start_list
df['date_stop'] = insights_date_stop_list

In [ ]:
df.head()

In [ ]:
df.groupby(['campaign_id']).sum()